In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


# df_tweet_US
"dataUSTrump_clean_sentiment.csv"(2020-01-01 ~ 2021-01-20) + "dataUSBiden_sentiment.csv" (2021-01-20 ~ 2022-04-09)  
distilBert

In [ ]:
def formatize(df_tweet):
  # remove any rows that are just the row titles
  df_tweet = df_tweet[df_tweet['created_at'] != 'created_at']

  # type conversion
  df_tweet['created_at'] = pd.to_datetime(df_tweet['created_at'])

  # sort
  df_tweet = df_tweet.sort_values(by='created_at')

  return df_tweet

In [ ]:
# read CSV
path = "drive/MyDrive/Colab Notebooks/ADS/USdata/"
df_Trump_raw = pd.read_csv(path +"dataUSTrump_clean_sentiment.csv")
df_Biden_raw = pd.read_csv(path +"dataUSBiden_sentiment.csv")

df_Biden = formatize(df_Biden_raw)
df_Trump = formatize(df_Trump_raw)

# filter out opinion polls taken before or after our tweet window
partition = datetime.strptime("2021-01-21 00:00:00", '%Y-%m-%d %H:%M:%S')

df_us_Biden = df_Biden.loc[(partition < df_Biden.created_at)]
df_us_Trump = df_Trump.loc[(df_Trump.created_at < partition)]

df_us_data = df_us_Trump.append(df_us_Biden)

df_us_data.set_index('created_at', inplace=True)
df_us_data["distilbert_weight"] = df_us_data["distilbert_label"]*df_us_data["distilbert_score"]


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (0,3,4,5,6,7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:

df_us = df_us_data['distilbert_weight'].to_frame()
df_us = df_us.reset_index()
df_us.rename(columns={'distilbert_weight': 'sentiment'},inplace=True)
df_us['sentiment_normalised'] =  (df_us['sentiment'] - df_us['sentiment'].min()) / (df_us['sentiment'].max() - df_us['sentiment'].min())
df_us['created_at'] = pd.to_datetime(df_us['created_at'])


In [ ]:
df_us

,created_at,sentiment,sentiment_normalised
0,2020-01-01 00:00:53,-0.994309,0.002756
1,2020-01-01 00:00:53,-0.994309,0.002756
2,2020-01-01 00:03:30,0.985849,0.992977
3,2020-01-01 00:03:30,0.985849,0.992977
4,2020-01-01 00:05:05,-0.997296,0.001262
...,...,...,...
369138,2022-04-09 23:23:37,0.952685,0.976393
369139,2022-04-09 23:24:40,-0.995401,0.002210
369140,2022-04-09 23:24:40,-0.995401,0.002210
369141,2022-04-09 23:50:32,0.999357,0.999732


In [ ]:
df_us.to_csv(path + 'df_tweet_US.csv', encoding='utf-8', index=False)

# df_opinion_US ( opinion us -> df opinion us )  
2020/01/01 ~ 2022/04/20  
date, opinion, opinion_normalised


In [ ]:
# def remove_title(df, column):
#   df = df[df[column] != column]

# def pds_to_datetime(df, column):
#   df[column] = pd.to_datetime(df[column])
  
# def index(df, column):
#   df.set_index(column,inplace=True)


In [ ]:
def format_opinion(df_opinion):
  # select polls for all polls (among All polls, Adults, Voters)
  df_opinion = df_opinion.loc[df_opinion['subgroup'] == 'All polls']
  
  # calculate net approval approve - disapprove
  df_opinion['net_approval'] = df_opinion['approve_estimate'] - df_opinion['disapprove_estimate']
  
  # drop irrelevant features
  df_opinion = df_opinion.drop(['president', 'subgroup', 'approve_estimate', 'approve_hi', 'approve_lo', 'disapprove_estimate', 'disapprove_hi', 'disapprove_lo',  'timestamp'], axis =1)
  
  

  df_opinion.rename(columns={'modeldate': 'date'}, inplace = True) # rename column to date
  # # type conversion
  df_opinion['date'] = pd.to_datetime(df_opinion['date'])
  # # filter out opinion polls taken before or after our tweet window
  datetime_start = datetime.strptime("2019-12-31 00:00:00", '%Y-%m-%d %H:%M:%S')
  datetime_end = datetime.strptime("2022-04-21 00:00:00", '%Y-%m-%d %H:%M:%S')
  df_opinion = df_opinion.loc[(datetime_start < df_opinion['date']) & (df_opinion['date'] < datetime_end)]

  df_opinion.rename(columns={"net_approval": 'opinion'}, inplace = True) # rename column to opinion
  df_opinion['opinion_normalised'] =  (df_opinion['opinion'] - df_opinion['opinion'].min()) / (df_opinion['opinion'].max() - df_opinion['opinion'].min())
  df_opinion['date'] = pd.to_datetime(df_opinion['date'])
  return df_opinion

In [ ]:
path = "drive/MyDrive/Colab Notebooks/ADS/USdata/"
# load opinion poll data
df_us_poll_data = pd.read_csv(path + "opinionUS.csv")
df_us_poll = format_opinion(df_us_poll_data)

# df_us_poll = df_us_poll.reset_index()
df_us_poll

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,date,opinion,opinion_normalised
3220,2020-01-01,-10.261105,0.244969
3222,2020-01-02,-10.672149,0.234619
3225,2020-01-03,-10.465896,0.239812
3229,2020-01-04,-10.465896,0.239812
3232,2020-01-05,-10.465896,0.239812
...,...,...,...
5720,2022-04-16,-10.594049,0.236585
5723,2022-04-17,-10.594049,0.236585
5726,2022-04-18,-10.096649,0.249110
5727,2022-04-19,-10.059837,0.250037


In [ ]:

# # df_trump_poll = df_us_poll.loc[df_us_poll_data['president'] == 'Donald Trump'] # 2020-01-01 ~ 2021-01-20 (386)
# # df_joeBiden_poll = df_us_poll.loc[df_us_poll_data['president'] == 'Joseph R. Biden Jr.'] #2021-01-23 ~ 2021-10-26 (277)
# df_biden_poll = df_us_poll.loc[df_us_poll_data['president'] == 'Joe Biden'] # 2021-10-27 ~ 2022-04-20 (175)
# # 2021-01-21, 22, 2022-03-19 missing

In [ ]:
df_us_poll.to_csv(path + 'df_opinion_US.csv', encoding='utf-8', index=False)